(In order to load the stylesheet of this notebook, execute the last code cell in this notebook)

# Recommender System for Amazon Electronics

In this assignment, we will be working with the [Amazon dataset](http://cs-people.bu.edu/kzhao/teaching/amazon_reviews_Electronics.tar.gz). You will build a recommender system to make predictions related to reviews of Electronics products on Amazon.

Your grades will be determined by your performance on the predictive tasks as well as a brief written report about the approaches you took.

This assignment should be completed **individually**.

## Files

**train.json** 1,000,000 reviews to be used for training. It is not necessary to use all reviews for training if doing so proves too computationally intensive. The fields in this file are:

* **reviewerID** The ID of the reviewer. This is a hashed user identifier from Amazon.

* **asin** The ID of the item. This is a hashed product identifier from Amazon.

* **overall** The rating of reviewer gave the item.

* **helpful** The helpfulness votes for the review. This has 2 subfields, 'nHelpful' and 'outOf'. The latter is the total number of votes this review received. The former is the number of those that considered the review to be helpful.

* **reviewText** The text of the review.

* **summary** The summary of the review.

* **unixReviewTime** The time of the review in seconds since 1970.

**meta.json** Contains metadata of the items:

* **asin** The ID of the item.

* **categories** The category labels of the item being reviewed.

* **price** The price of the item.

* **brand** The brand of the item.

**pairs_Rating.txt** The pairs (reviewerID and asin) on which you are to predict ratings.

**pairs_Purchase.txt** The pairs on which you are to predict whether a user purchased an item or not.

**pairs_Helpful.txt** The pairs on which you are to predict helpfulness votes. A third column in this file is the total number of votes from which you should predict how many were helpful.

**helpful.json** The review data associated with the helpfulness prediction test set. The 'nHelpful' field has been removed from this data since that is the value you need to predict above. This data will only be of use for the helpfulness prediction task.

**baseline.py** A simple baseline for each task.

## Tasks

**Rating prediction** Predict people's star ratings as accurately as possible for those (reviewerID, asin) pairs in 'pairs_Rating.txt'. Accuracy will be measured in terms of the [root mean-squared error (RMSE)](http://www.kaggle.com/wiki/RootMeanSquaredError).

**Purchase prediction** Predict given a (reviewerID, asin) pair from 'pairs_Purchase.txt' whether the user purchased the item (really, whether it was one of the items they reviewed). Accuracy will be measured in terms of the [categorization accuracy](http://www.kaggle.com/wiki/HammingLoss) (1 minus the Hamming loss).

**Helpfulness prediction** Predic whether a user's review of an item will be considered helpful. The file 'pairs_Helpful.txt' contains (reviewerID, asin) pairs with a third column containing the number of votes the user's review of the item received. You must predict how many of them were helpful. Accuracy will be measured in terms of the total [absolute error](http://www.kaggle.com/wiki/AbsoluteError), i.e. you are penalized one according to the difference |nHelpful - prediction|, where 'nHelpful' is the number of helpful votes the review actually received, and 'prediction' is your prediction of this quantity.

We set up competitions on Kaggle to keep track of your results compared to those of other members of the class. The leaderboard will show your results on half of the test data, but your ultimate score will depend on your predictions across the whole dataset.
* Kaggle competition: [rating prediction](https://inclass.kaggle.com/c/cs591-hw3-rating-prediction3) click here to [join](https://kaggle.com/join/datascience16rating)
* Kaggle competition: [purchase prediction](https://inclass.kaggle.com/c/cs591-hw3-purchase-prediction) click here to [join](https://kaggle.com/join/datascience16purchase)
* Kaggle competition: [helpfulness prediction](https://inclass.kaggle.com/c/cs591-hw3-helpful-prediction2) click here to

In [2]:
from collections import defaultdict

def readJson(f):
    for l in open(f):
        yield eval(l)

In [28]:
#####################################
#predictions_Rating
#####################################

allRatings = []
userRatings = defaultdict(list)
itemRatings = defaultdict(list)
# read ratings for each user into dict
for l in readJson('./amazon_reviews_Electronics/train.json'):
    user,item,helpful = l['reviewerID'],l['asin'],l['helpful']['nHelpful']
    allRatings.append(l['overall'])
    userRatings[user] += [l['overall']] #* (helpful + 1)
    itemRatings[item] += [l['overall']] #* (helpful + 1)

globalAverage = sum(allRatings) / len(allRatings)
userAverage = {}
itemAverage = {}
# calculate avg ratings for each user
for u in userRatings:
    userAverage[u] = sum(userRatings[u]) / len(userRatings[u])
for i in itemRatings:
    itemAverage[i] = sum(itemRatings[i]) / len(itemRatings[i])

count = 0
count_ui = 0
count_u = 0
count_i = 0
#predict ratings according to pairs
predictions = open("./amazon_reviews_Electronics/predictions_Rating.txt", 'w')
for l in open("./amazon_reviews_Electronics/pairs_Rating.txt"):
    if l.startswith("reviewerID"):
        #header
        predictions.write(l)
        continue
    u,i = l.strip().split('-')
    if u in userAverage:
        if i in itemAverage:
            count_ui += 1
            #30-40-30
            predictions.write(u + '-' + i + ',' + str(userAverage[u] * 0.3 + itemAverage[i] * 0.4 + globalAverage * 0.3) + '\n')
        else:
            #20-80
            count_u += 1
            predictions.write(u + '-' + i + ',' + str(userAverage[u] * 0.2 + globalAverage * 0.8) + '\n')
    elif i in itemAverage:
        #50-50
        count_i += 1
        predictions.write(u + '-' + i + ',' + str(itemAverage[i] * 0.5 + globalAverage * 0.5) + '\n')
    else:
        count += 1
        predictions.write(u + '-' + i + ',' + str(globalAverage - 0.2) + '\n')
print count, count_ui, count_u, count_i
predictions.close()

1964 68802 5474 23760


predictions_Rating:
* for rating prediction, first I calculated item average based on training data, then I combine useraverage, itemaverage and global average as much as possible.  For example, some user-item pair might not have useraverage or itemaverage data, in this situation I just combine the data I have.  For those u-i pair that has neither itemaverage nor useraverage data, I still used globalaverage data, but based on the rule of competition, I calibrated globalAverage so that it will match the target dataset better.

In [ ]:
#####################################
#predictions_Purchase
#####################################

itemCount = defaultdict(int)
totalPurchases = 0
priceDict = {}
#read perchase data for item, count how many time it's purchased
for l in readJson('./amazon_reviews_Electronics/train.json'):
    user,item = l['reviewerID'],l['asin']
    itemCount[item] += int(l['overall'])
    totalPurchases += int(l['overall'])
# for line in readJson('./amazon_reviews_Electronics/meta.json'):
#     if line['price'] == '':
#         priceDict[line['asin']] = -1.0
#     else:
#         priceDict[line['asin']] = float(line['price'])

#sort itemCount
mostPopular = [(itemCount[x], x) for x in itemCount]
mostPopular.sort()
print mostPopular
mostPopular.reverse()
print '==============================================================\n\n'
print mostPopular
#
return1 = set()
count = 0
#for itemCount, item in mostPopular
for ic, i in mostPopular:
    count += ic
    return1.add(i)
    if count > totalPurchases * 0.9: 
        break
# predict according to purchase record
count = 0
predictions = open("./amazon_reviews_Electronics/predictions_Purchase.txt", 'w')
for l in open("./amazon_reviews_Electronics/pairs_Purchase.txt"):
    if l.startswith("reviewerID"):
        #header
        predictions.write(l)
        continue
    u,i = l.strip().split('-')
    if i in return1:
        predictions.write(u + '-' + i + ",1\n")
    else:
        predictions.write(u + '-' + i + ",0\n")
print count
predictions.close()

predictions_Purchase:
* I don't have a very good idea about this task, I've tried to predict cheaper items and high rating items as purchased but it does no better than baseline output.  After correcting the reversed mostpopular items i just change the break point to count > 0.9 * total purchase and it gets better.  Brand could be a useful information but since it's removed so i'm not using it here. 

In [37]:
#####################################
#predictions_Helpful
#####################################

allHelpful = []
userHelpful = defaultdict(list)
itemRatings = defaultdict(list)
itemAverage = defaultdict(list)
# read helpful data for each users into dict
for l in readJson('./amazon_reviews_Electronics/train.json'):
    user,item = l['reviewerID'],l['asin']
    allHelpful.append(l['helpful'])
    userHelpful[user].append(l['helpful'])
    itemRatings[item] += [l['overall']]
    
# review length and rating for u-i pair
helpfulDict = {}
for l in readJson('./amazon_reviews_Electronics/helpful.json'):
    useritem, rating = l['reviewerID'] + '-' + l['asin'], float(l['overall'])
    text_len = len(l['reviewText'])
    helpfulDict[useritem] = {'rating':rating, 'text_len':text_len}

# calculate avg item ratings    
for i in itemRatings:
    itemAverage[i] = sum(itemRatings[i]) / len(itemRatings[i])

#calculate global avg
averageRate = sum([x['nHelpful'] for x in allHelpful]) * 1.0 / sum([x['outOf'] for x in allHelpful])
userRate = {}

#calculate avg helpful for each user(out of 1)
for user in userHelpful:
    userRate[user] = sum([x['nHelpful'] for x in userHelpful[user]]) * 1.0 / sum([x['outOf'] for x in userHelpful[user]])

#predict helpful according to user avg helpful
count = 0
predictions = open("./amazon_reviews_Electronics/predictions_Helpful.txt", 'w')
for l in open("./amazon_reviews_Electronics/pairs_Helpful.txt"):
    if l.startswith("reviewerID"):
        #header
        predictions.write(l)
        continue
    u,i,outOf = l.strip().split('-')
    useritem = u + '-' + i
    outOf = int(outOf)
    if outOf == 1:
        predictions.write(u + '-' + i + '-' + str(outOf) + ',' + '1.0' + '\n')
    elif i in itemRatings:
        tempRate = 1.0
        if abs(helpfulDict[useritem]['rating'] - itemAverage[i]) > 1.5:
            count += 1
            tempRate = 0.8
        if abs(helpfulDict[useritem]['rating'] - itemAverage[i]) > 3.0:
            tempRate = 0.5
        if helpfulDict[useritem]['text_len'] > 2000 or outOf > 50:
            tempRate = 0.95
        predictions.write(u + '-' + i + '-' + str(outOf) + ',' + str(outOf*tempRate) + '\n')
    elif u in userRate:
        predictions.write(u + '-' + i + '-' + str(outOf) + ',' + str(outOf*userRate[u]) + '\n')
    else:
        #use global if no reference
        predictions.write(u + '-' + i + '-' + str(outOf) + ',' + str(outOf*averageRate) + '\n')
print count
predictions.close()

17733


predictions_Helpful:
* In this task i calculated itemaverage and read the user's rating in helpful.json and compare the user's rating and the average rating of the item.  I think with the different gets bigger less people will likely to consider his review as useful.  Also I find out the more people reviewed the user's review the more would be "helpful".  And I also predicted those reviews with a text length more than 2000 would receive most of the reviews as "helpful", since it's not easy to write all these just for a online shopping review(At least i would give him a thumbs up).  For those items that has no item average, I used user average for that and if there is neither uesr average nor item average, I just use the global average for that.

## Grading and Evaluation

You will be graded on the following aspects.

* Your written report. This should describe the approaches you took to each of the 3 tasks. To obtain good performance, you should not need to invent new approaches (though you are more than welcome to) but rather you will be graded based on your decision to apply reasonable approaches to each of the given tasks. (**10pts** for each task)

* Your ability to obtain a solution which outperforms the baselines on the unseen portion of the test data. Obtaining full marks requires a solution which is substantially better (at least several percent) than baseline performance. (**10pts** for each task)

* Your ranking for each of the three tasks compared to other students in the class. (**5pts** for each task)

* Obtain a solution which outperforms the baselines on the seen portion of the test data (the leaderboard). 
(**5pts** for each task)

## Baselines

Simple baselines have been provided for each of the 3 tasks. These are included in 'baselines.py' among the files above. These 3 baselines operate as follows:

**Rating prediction** Returns the global average rating, or the user's average if you have seen them before in the training data.

**Purchase prediction** Finds the most popular products that account for 50% of purchases in the training data. Return '1' whenever such a product is seen at test time, '0' otherwise.

** Helpfulness prediction** Multiplies the number of votes by the global average helpfulness rate, or the user's rate if we saw this user in the training data.

Running 'baseline.py' produces 3 files containing predicted outputs. Your submission files should have the same format.

## Dataset Citation

**Image-based recommendations on styles and substitutes** J. McAuley, C. Targett, J. Shi, A. van den Hengel *SIGIR*, 2015

**Inferring networks of substitutable and complementary products** J. McAuley, R. Pandey, J. Leskovec *Knowledge Discovery and Data Mining*, 2015

-----------------

In [ ]:
# Code for setting the style of the notebook
from IPython.core.display import HTML
def css_styling():
    styles = open("../theme/custom.css", "r").read()
    return HTML(styles)
css_styling()